In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
# Configs
%config InlineBackend.figure_format ='retina'
%matplotlib inline

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn import externals, base, neural_network, neighbors, feature_extraction, datasets, linear_model, metrics, model_selection, preprocessing, svm, ensemble, decomposition, feature_extraction, utils

import keras
import scipy

Using TensorFlow backend.


In [76]:
# Load Bridge helper classes
%run helpers/bridge.ipynb
%run helpers/load-data.ipynb
%run helpers/train-eval-utils.ipynb

In [77]:
myd = Deal()

In [78]:
myd.pbn

'A9654.5.AKQ63.K5 QT732.A93.T98.T7 K.QJT874.J4.Q843 J8.K62.752.AJ962'

In [79]:
myd.wnrs

[{'N': 8, 'E': 8, 'S': 7, 'W': 8},
 {'N': 8, 'E': 8, 'S': 8, 'W': 8},
 {'N': 9, 'E': 10, 'S': 10, 'W': 10},
 {'N': 10, 'E': 10, 'S': 10, 'W': 10},
 {'N': 8, 'E': 8, 'S': 7, 'W': 8}]

In [80]:
myd.solve().wnrs

[{'S': 8.0, 'E': 9.0, 'N': 8.0, 'W': 9.0},
 {'S': 8.0, 'E': 8.0, 'N': 8.0, 'W': 8.0},
 {'S': 10.0, 'E': 10.0, 'N': 10.0, 'W': 10.0},
 {'S': 9.0, 'E': 10.0, 'N': 9.0, 'W': 10.0},
 {'S': 7.0, 'E': 8.0, 'N': 7.0, 'W': 8.0}]

In [83]:
import cProfile
cProfile.run('foo()')


         9978743 function calls (9946743 primitive calls) in 12.006 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    52000    0.018    0.000    0.018    0.000 <ipython-input-76-078efab8578b>:36(pos)
   183645    0.065    0.000    0.065    0.000 <ipython-input-76-078efab8578b>:43(__lt__)
    52000    0.142    0.000    0.204    0.000 <ipython-input-76-078efab8578b>:7(__new__)
    20000    0.282    0.000    0.282    0.000 <ipython-input-76-2e4ff5113e48>:1(round_p)
   280000    0.452    0.000    0.466    0.000 <ipython-input-76-3507402eb63e>:1(score_contract)
    16000    0.017    0.000    0.017    0.000 <ipython-input-76-bda48e7a1639>:100(<listcomp>)
   210000    0.469    0.000    1.041    0.000 <ipython-input-76-bda48e7a1639>:125(comp_score)
     1000    0.168    0.000    1.209    0.001 <ipython-input-76-bda48e7a1639>:139(fill_score)
     1000    0.479    0.000    0.874    0.001 <ipython-input-76-bda48e7a1639>:147(compute_

In [82]:
def foo():
    # 20x speedup if you don't use original DDS
    for _ in range(1000):
        Deal()

# Helpers

In [716]:
def gen_sequence(deal, s1, s2=System(pass_function), s3=None, s4=None):
    seq = Bidding()
    if s3 is None:
        s3 = s1
    if s4 is None:
        s4 = s2
    sgs = [s1, s2, s3, s4]
    while not seq.done:
        cur_pl = len(seq) % 4
        seq = sgs[cur_pl].step(deal.hands[cur_pl], seq)
    return seq

# Examples

In [692]:
# Full knowledge ~ Theoretical limit

In [864]:
X = np.array([d.N.feature_array() + d.S.feature_array() for d in data] * no_samples) #only N hand


In [693]:
# TODO

In [872]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(256, 256, 256, 256,), 
                                     max_iter=5, 
                                     #early_stopping=True, tol=1e-6
                                    )

In [873]:
model.fit(X, y) # 3min


/Applications/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(256, 256, 256, 256), learning_rate='constant',
              learning_rate_init=0.001, max_iter=5, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [874]:
model.score(X[:10000], y[:10000])

0.3764

In [875]:
System(system_from_model(model)).evaluate(data[:1000])

-6.949

## One hand places the contract

### Basic things

In [694]:
def const_function(contr):
    def fn(hand, bidding):
        return contr
    return fn

In [719]:
def pass_function(hand, bidding):
    return 'PASS'

In [696]:
def hcp_function(start=14.2, step=3.9):
    def fn(hand, bidding):
        ft = hand.features()
        if ft['hcp'] >= start:
            lev = min(7, (ft['hcp'] - start) / step + 1)
            return str(int(lev)) + 'NT'
        return 'PASS'
    return fn

In [697]:
def suit_function(start=12.8, step=2.6):
    def fn(hand, bidding):
        ft = hand.features()
        if ft['hcp'] >= start:
            lev = min(7, (ft['hcp'] - start) / step + 1)
            strain = 'NT'
            bst_len = 4
            for s in suits:
                if ft[s + '_len'] >= bst_len:
                    strain = s
                    bst_len = ft[s + '_len']
            return str(int(lev)) + strain
            #return const_function(str(int(lev)) + strain)(hand, bidding)
        return 'PASS'
    return fn

In [698]:
def system_from_model(model):
    def fn(hand, bidding):
        bid = model.predict(hand.feature_array().reshape(1, -1))[0]
        return bid
    return fn

### Theoretical limit

In [ ]:
# Try to make a NN learn the best contract with_opps

In [839]:
no_samples = 5 # 2 min per sample

X = np.array([d.hands[0].feature_array() for d in data] * no_samples) #only N hand

# Winners in a particular strain played from N
y = []
for _ in range(no_samples):
    y += [contract_to_index[d.sample_contract()]
     for d in data]

In [840]:
# This model overfits terribly

In [844]:
model = neural_network.MLPClassifier(hidden_layer_sizes=(256, 256, 256, 256,), 
                                     max_iter=25, 
                                     #early_stopping=True, tol=1e-6
                                    )

In [845]:
# 1 iter = 2.5 mins
model.fit(X, y) # 3min
print(model.n_iter_)

22


/Applications/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [854]:
from sklearn.externals import joblib
joblib.dump(model, 'models/ftarrayN_to_bestc.pkl') 

['models/ftarrayN_to_bestc.pkl']

In [572]:
# Load the pickle file
model_load = joblib.load('models/ftarrayN_to_bestc.pkl') 

0.96585

In [865]:
# Care: it's in sample
System(system_from_model(model)).evaluate(data[:1000])

-4.286

## One hand describes, other places

In [754]:
def basic_function(hand, bidding):
    bids_made = len(bidding)
    ft = hand.features()
    
    min_opening_str = 10
    hcp_step = 4
    card_fit_suit = 8
    
    # if other hand has not described
    if bids_made <= 1:
        if ft['hcp'] < min_opening_str:
            return 'PASS'
        else:
            # [10, 13] [14, 17] ...
            lev = min(7, (ft['hcp'] - min_opening_str) / hcp_step + 1)
            # Bids a 5 card suit if it has it, else NT
            strain = 'NT'
            bst_len = 5
            for s in suits:
                if ft[s + '_len'] >= bst_len:
                    strain = s
                    bst_len = ft[s + '_len']
            return str(int(lev)) + strain
        
    # if other hand has described
    if 1 < bids_made <= 3:
        p_bid = bidding.partner_bids()[0]
        # Estimates hcp
        if p_bid == 'PASS':
            ft['hcp'] += min_opening_str - 3
        else:
            ft['hcp'] += min_opening_str - 3 + hcp_step * int(p_bid[0])
        # Estimates distribution as 3333 or 5222
        if p_bid == 'PASS' or p_bid[1:] == 'NT':
            for s in suits:
                ft[s + '_len'] += 3
        else:
            for s in suits:
                if s == p_bid[1:]:
                    ft[s + '_len'] += 5
                else:
                    ft[s + '_len'] += 2
        # Guesses best contract
        # strain = s if >= 8 card fit, else NT
        strain = 'NT'
        bst_len = card_fit_suit
        for s in suits:
            if ft[s + '_len'] >= bst_len:
                strain = s
                bst_len = ft[s + '_len']
                
        # level for NT 19, 22, 25, 28, 31, 34, 37
        if strain == 'NT':
            level = (ft['hcp'] - 16) // 3
        # level for suit depends on length
        else:
            # hcp + 2*length >= 25 + 16 for game
            level = (ft['hcp'] + 2 * ft[strain + '_len'] - 29) // 3
        level = min(7, level)

        # Principle of suit correction:
        #  if p made a bid in a NT strain we might wish to raise the level
        #  to change that to a suit
        if p_bid == '1NT' or p_bid == '2NT':
            if strain != 'NT' and level <= int(p_bid[0]):
                level = int(p_bid[0]) + 1
            
        # for minor suits, correct 4m and 5m to 3NT
        if strain in ['c', 'd'] and 4 <= level <= 5:
            level = 3; strain = 'NT'
        # correct non-slams above game
        if strain in ['h', 's'] and level == 5:
            level = 4
        if strain == 'NT' and 4 <= level <= 5:
            level = 3


        if level <= 0:
            return 'PASS'
        else:
            return str(level) + strain
    
    # if other hand has placed the contract
    if bids_made >= 4:
        return 'PASS'

# Evaluations

In [828]:
System(pass_function).evaluate(data)

-4.547764336291191

In [671]:
System(hcp_function()).evaluate(data)

-4.334017286610689

In [672]:
System(suit_function()).evaluate(data)

-4.248439517422304

In [866]:
# NN that takes North hand and estimates a contract (IN SAMPLE)
model_load = joblib.load('models/ftarrayN_to_bestc.pkl') 
System(system_from_model(model_load)).evaluate(data[:10000])

-4.673

In [673]:
System(basic_function).evaluate(data)

-3.595602687566305

In [755]:
System(basic_function).compare(data[:1000], System(suit_function()))

1.011